# Taranaki Annual Seismic and Geodetic Report for TRC

## Purpose:
Streamline production of data and figures for Taranaki TRC anuual report. All what is needed in a single notebook.

This is not highly documented as it is not intended to be for the 'casual user'.

All we need is a notebook to write the report!!

## Problems:
With Python3 version, I can't seem to plot GNSS error bars.

A recent version of Matplotlib is extending aces beyond data limits. set_xlim etc allows this to be fixed, but not for GNSS time series plots.

## Author:
Steven Sherburn

## Date:
July 2017

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator
from matplotlib.transforms import offset_copy
from matplotlib import patheffects


import scipy.signal
import numpy as np
from math import floor

import datetime
from dateutil import parser

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.io.img_tiles import OSM
from cartopy.io.img_tiles import Stamen

import warnings; warnings.simplefilter('ignore')

In [ ]:
#box for search area
bbox = '173.5,-39.75,174.58,-38.83'
lonmin = 173.5
lonmax = 174.58
latmin = -39.75
latmax = -38.83

In [ ]:
startall = '1994-01-01'
startyear = '2019-07-01'
end = '2020-07-01'

Seismic
--

**Seismic data since 1994**

In [ ]:
url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&startdate='+startall+'&enddate='+end

In [ ]:
eqs = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')

**NZ Active Faults Database**

In [ ]:
#use existing GMT format file with '>' indicating a line break
names=['longitude', 'latitude']
faults = pd.read_csv('nzafd_250k.psxy', sep='\s+', na_values='>', names=names)

Maps
--

**This year**

In [ ]:
#number of events is the year
eqs['publicid'][(eqs.index>startyear)&(eqs.index<end)].count()

In [ ]:
#prepare eqs
eqsyr50 = eqs[(eqs.index>startyear)&(eqs.index<end)&(eqs[' depth']<50)]
eqsyrdp = eqs[(eqs.index>startyear)&(eqs.index<end)&(eqs[' depth']>=50)]

In [ ]:
#cross-section area
slatmin = -39.083
slatmax = -39.417
slonmin = 173.51
slonmax = 174.567
boxlon = [slonmin, slonmin, slonmax, slonmax, slonmin]
boxlat = [slatmin, slatmax, slatmax, slatmin, slatmin]

In [ ]:
#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)
# gl = ax.gridlines(draw_labels=True, alpha = 0.5)
# gl.xlabels_top = False
# gl.ylabels_right = False

#active faults
plt.plot(faults['longitude'], faults['latitude'], color='black', transform=ccrs.Geodetic())

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(eqsyr50['longitude'], eqsyr50[' latitude'], color='red', marker='o', s=eqsyr50[' magnitude']**2, transform=ccrs.PlateCarree(), label='<50')
plt.scatter(eqsyrdp['longitude'], eqsyrdp[' latitude'], facecolors='None', edgecolors='black', linewidth=0.5, marker='o', s=eqsyrdp[' magnitude']**2, transform=ccrs.PlateCarree(), label=r'$\geq$'+'50')

#section position
plt.plot(boxlon, boxlat, color='black', linestyle='--', marker='None', transform=ccrs.Geodetic())

plt.legend(loc='best', handletextpad=0, prop={'size': 6})

#scale_bar(ax, ccrs.Mercator(), 20)
# plt.tight_layout()
plt.savefig('year_map.png', dpi=400)

**Events M>x, to help identify bigger ones**

In [ ]:
yrm3 = eqsyr50[eqsyr50[' magnitude']>3]
yrm3

In [ ]:
#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)

#active faults
plt.plot(faults['longitude'], faults['latitude'], color='black', transform=ccrs.Geodetic())

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(yrm3['longitude'], yrm3[' latitude'], color='red', marker='o', s=yrm3[' magnitude']**2, transform=ccrs.PlateCarree())

**All data, since 1994**

In [ ]:
#prepare eqs
eqsall50 = eqs[eqs[' depth']<50]
eqsalldp = eqs[eqs[' depth']>=50]

In [ ]:
eqs[(eqs[' depth']<50)&(eqs[' magnitude']>=4.5)]

In [ ]:
#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)
#ax.gridlines()

#active faults
plt.plot(faults['longitude'], faults['latitude'], color='black', transform=ccrs.Geodetic())

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(eqsall50['longitude'], eqsall50[' latitude'], color='red', linewidths=0, marker='o', alpha=0.25, s=eqsall50[' magnitude']**2, transform=ccrs.PlateCarree(), label='<50')
plt.scatter(eqsalldp['longitude'], eqsalldp[' latitude'], facecolors='None', edgecolors='black', linewidth=0.5, marker='o', s=eqsalldp[' magnitude']**2, transform=ccrs.PlateCarree(), label=r'$\geq$'+'50')

plt.legend(loc='best', handletextpad=0, prop={'size': 6})

# plt.tight_layout() 
plt.savefig('all_map.png', dpi=400)

**All data, since 1994, magnitude >= 2.7**

In [ ]:
#prepare eqs
eqsall5027 = eqs[(eqs[' depth']<50)&(eqs[' magnitude']>=2.7)]
eqsalldp27 = eqs[(eqs[' depth']>=50)&(eqs[' magnitude']>=2.7)]

In [ ]:
#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 10)
#ax.gridlines()

#active faults
plt.plot(faults['longitude'], faults['latitude'], color='black', transform=ccrs.Geodetic())

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
plt.scatter(eqsall5027['longitude'], eqsall5027[' latitude'], color='red', marker='o', s=eqsall5027[' magnitude']**2, transform=ccrs.PlateCarree(), label='<50')
plt.scatter(eqsalldp27['longitude'], eqsalldp27[' latitude'], facecolors='None', edgecolors='black', linewidth=0.5, marker='o', s=eqsalldp27[' magnitude']**2, transform=ccrs.PlateCarree(), label=r'$\geq$'+'50')

plt.legend(loc='best', handletextpad=0, prop={'size': 6}, framealpha=1)

# plt.tight_layout() 
plt.savefig('all_2.7_map.png', dpi=400)

**North Island Map for this year**

In [ ]:
#box for search area
wlatmin = -40.5
wlatmax = -38.1
wlonmin = 172.6
wlonmax = 177.3

bbox = '172.6,-40.5,177.3,-38.1'
url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&startdate='+startyear+'&enddate='+end

eqsw = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')

In [ ]:
#prepare eqs
eqsw10 = eqsw[eqsw[' depth']<10]
eqsw10_20 = eqsw[(eqsw[' depth']>=10)&(eqsw[' depth']<20)]
eqsw20_30 = eqsw[(eqsw[' depth']>=20)&(eqsw[' depth']<30)]
eqsw30_50 = eqsw[(eqsw[' depth']>=30)&(eqsw[' depth']<50)]
eqsw50 = eqsw[eqsw[' depth']>=50]

In [ ]:
#area of main plot
mlon = [lonmin, lonmin, lonmax, lonmax, lonmin]
mlat = [latmin, latmax, latmax, latmin, latmin]

In [ ]:
#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([wlonmin, wlonmax, wlatmin, wlatmax])
ax.add_image(imagery, 8)
#ax.gridlines()

#hypocentres, symbol size=magnitude squared, as scatter symbol size is symbol area, square gives 'pleasing' image
#plotting order so that shallow symbols overly deeper
plt.scatter(eqsw50['longitude'], eqsw50[' latitude'], color='black', marker='o', s=eqsw50[' magnitude']**2, transform=ccrs.PlateCarree(), label=r'$\geq$'+'50')
plt.scatter(eqsw30_50['longitude'], eqsw30_50[' latitude'], color='blue', marker='o', s=eqsw30_50[' magnitude']**2, transform=ccrs.PlateCarree(), label='30-50')
plt.scatter(eqsw20_30['longitude'], eqsw20_30[' latitude'], color='red', marker='o', s=eqsw20_30[' magnitude']**2, transform=ccrs.PlateCarree(), label='20-30')
plt.scatter(eqsw10_20['longitude'], eqsw10_20[' latitude'], color='green', marker='o', s=eqsw10_20[' magnitude']**2, transform=ccrs.PlateCarree(), label='10-20')
plt.scatter(eqsw10['longitude'], eqsw10[' latitude'], color='yellow', marker='o', s=eqsw10[' magnitude']**2, transform=ccrs.PlateCarree(), label='<10')

#main plot area
plt.plot(mlon, mlat, color='black', linestyle='--', marker='None', transform=ccrs.Geodetic())

plt.legend(loc='best', handletextpad=0)

plt.tight_layout() 
plt.savefig('year_wide_map.png', dpi=400)

**Map of basement faults - for presentation reference only**

In [ ]:
#use existing GMT format file with '>' indicating a line break
names=['longitude', 'latitude', 'mE', 'mN']
tarfaults1 = pd.read_csv('taranaki_faults-1.gmt', sep='\s+', na_values='>', names=names)
tarfaults2 = pd.read_csv('taranaki_faults-2.gmt', sep='\s+', na_values='>', names=names)

In [ ]:
#imagery
imagery = OSM()
# imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent([lonmin, lonmax, latmin, latmax])
ax.add_image(imagery, 8)
#ax.gridlines()

#
plt.plot(tarfaults1['longitude'], tarfaults1['latitude'], color='black', transform=ccrs.Geodetic())
plt.plot(tarfaults2['longitude'], tarfaults2['latitude'], color='black', transform=ccrs.Geodetic())

plt.tight_layout() 
plt.savefig('basement_faults_map.png', dpi=200)

**Histogram, monthly events, depth < 50 km**

In [ ]:
eqs50 = eqs[eqs[' depth']<50]
eqs50m3 = eqs[(eqs[' depth']<50)&(eqs[' magnitude']>=3)]
eqs50m4 = eqs[(eqs[' depth']<50)&(eqs[' magnitude']>=4)]

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3,ncols=1, sharex=True, figsize=(15,10))

mcount = eqs50m4.resample('1M', label='left', closed='left').count()
ax1.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax1.bar(mcount.index, mcount.publicid, width = 5, color='blue', edgecolor='blue', align='edge', label='M>=4 earthquakes')
ax1.set_xlim([startall,end])
ax1.legend(loc='upper left')
#want integer ticks(grid), default for small y-range gives 0.5 spacing
spacing = 1
majorLocator = MultipleLocator(spacing)
ax1.yaxis.set_major_locator(majorLocator)

mcount = eqs50m3.resample('1M', label='left', closed='left').count()
ax2.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax2.bar(mcount.index, mcount.publicid, width = 5, color='green', edgecolor='green', align='edge', label='M>=3 earthquakes')

ax2.legend(loc='upper left')
mcount = eqs50.resample('1M', label='left', closed='left').count()
ax3.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax3.bar(mcount.index, mcount.publicid, width = 5, color='red', edgecolor='red', align='edge', label='all earthquakes')
ax3.legend(loc='upper left')
plt.tight_layout()
fig.savefig('taranaki_monthly.png', dpi=200)

**Cumulative number**

Make a column containing event number (should be a better way of doing this)

In [ ]:
eqs50.sort_index(ascending=True, inplace=True)
eqs50.reset_index(inplace=True)
eqs50['evnum'] = eqs50.index+1
eqs50.set_index('origintime', inplace=True, drop=True)

In [ ]:
#cumulative number maximum
nmax = eqs50['evnum'].iloc[-1]
#tmin = eqs50.index[1]
#tmax = eqs50.index[-1]

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
plt.plot(eqs50['evnum'], color='red')
ax.set_ylim([0,nmax])
ax.set_xlim([startall,end])
#mark change to Sc3 analysis
ax.axvline(x='2012-01-01', color='black', linestyle='--', alpha=0.7)
ax.text('2012-03-01', 300, 'Change to SC3 analysis', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=12)
ax.axvline(x='2018-12-15', color='black', linestyle='--', alpha=0.7)
ax.text('2019-01-15', 300, 'NGMC', verticalalignment='bottom', horizontalalignment='left', rotation='vertical', fontsize=12)
ax.tick_params(axis='y', colors='red')
ax.set_ylabel('cumulative number', color = 'red')
ax.xaxis.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
ax.yaxis.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)

ax2= ax.twinx()
eqs50['energy'] = pow(10,(1.44 * eqs50[' magnitude'] + 5.24))
eqs50['cumeng'] = eqs50['energy'].cumsum()
eqs50['cumeng'] = 100 * eqs50['cumeng'] / eqs50['cumeng'].max()
ax2.plot(eqs50['cumeng'], color='blue', marker='None')
ax2.set_ylim([0,100])
ax2.tick_params(axis='y', colors='blue')
ax2.set_ylabel('percent cumulative energy', color = 'blue')
plt.tight_layout()
fig.savefig('taranaki_cumulative.png', dpi=200)

**Current year, cross-section and other stuff**

In [ ]:
bbox = '173.5,-39.75,174.58,-38.83'
url = 'https://quakesearch.geonet.org.nz/csv?bbox='+bbox+'&startdate='+startyear+'&enddate='+end
eqs = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')

**Total number events, for report**

In [ ]:
len(eqs.index)

In [ ]:
#select eqs in depth range and latitude range
eqs50 = eqs[eqs[' depth']<50]
sect = eqs50[(eqs50[' latitude']>-39.4167)&(eqs50[' latitude']<-39.0833)]

In [ ]:
#read topography profile file, convert to km
names=['longitude', 'elevation']
topo = pd.read_csv('taranaki_cross_section.xy', sep='\s+', names=names, usecols=[0,2], index_col=False)
topo['elevation'] = (topo['elevation'] / -1000)

In [ ]:
#hypocentres and topography on one plot, depth histogram on another
fig = plt.figure(figsize=(15,10))
gridspec.GridSpec(3,11)

#hypocentres and topography
plt.subplot2grid((3,11), (0,0), colspan=8, rowspan=3)
plt.plot(topo['longitude'], topo['elevation'], color='green')
plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
plt.scatter(x=sect['longitude'], y=sect[' depth'], color='red')
plt.text(x=173.55, y=-4, s='A', verticalalignment='top', horizontalalignment='left', fontsize=14)
plt.xlim([173.5,174.58])
plt.ylim([-5,50])
plt.ylabel('Depth (km)')
plt.plot(173.75, -0.3, marker='v', color='blue') #inverted triangle at coast
plt.gca().invert_yaxis()
plt.tight_layout()

#depth histogram
plt.subplot2grid((3,11), (0,9), colspan=2, rowspan=3)
plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
plt.hist(sect[' depth'], bins=[0,5,10,15,20,25,30,35,40,45], color='red', orientation='horizontal')
plt.text(x=2, y=-4, s='B', verticalalignment='top', horizontalalignment='left', fontsize=14)
plt.ylim([-5,50])
plt.xlabel('Number of earthquakes')
plt.ylabel('Depth (km)')
plt.gca().invert_yaxis()

fig.savefig('taranaki_depth-profile_histogram.png', dpi=200)

**Earthquakes within certain distances of the summit of Mt Taranaki**

This is to help the TVSAG understand a little better what has been located close to the volcano. 
To do this, it is easier to use GeoNet's Web Features Service (WFS) as this offers more functionality in selecting events. https://wfs.geonet.org.nz/

In [ ]:
#the park area, 9.5 km from the summit
url = 'http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=geonet:quake_search_v1&outputFormat=csv&cql_filter=depth<50+AND+origintime>='+startall+'+AND+origintime<='+end+'+AND+DWITHIN(origin_geom,Point+(174.063+-39.296),9500,meters)'
park = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')
park.sort_index(ascending=True, inplace=True) # sort by origintime(index), so events in time order

#the coast area, 26.5 km from the summit
url = 'http://wfs.geonet.org.nz/geonet/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=geonet:quake_search_v1&outputFormat=csv&cql_filter=depth<50+AND+origintime>='+startall+'+AND+origintime<='+end+'+AND+DWITHIN(origin_geom,Point+(174.063+-39.296),26500,meters)'
coast = pd.read_csv(url, parse_dates=['origintime'], index_col='origintime')
coast.sort_index(ascending=True, inplace=True) # sort by origintime(index), so events in time order

In [ ]:
#plot
fig = plt.figure(figsize=(10, 10))

d1 = datetime.datetime.strptime(startall, "%Y-%m-%d")
d2 = datetime.datetime.strptime(end, "%Y-%m-%d")

# magnitude vs time
ax1 = fig.add_subplot(2, 1, 1)
ymax = 1.1 * park.magnitude.max()
ax1.set_ylim([0, ymax])
ax1.plot(park.magnitude, marker='o', color='red', linestyle='None', label='manual')
ax1.text(0.01, 0.98,'Within 9.5 km (ENP circular boundary)', ha='left', va='top', transform=ax1.transAxes, fontsize=14)
ax1.set_ylabel('Magnitude')

# magnitude vs time
ax2 = fig.add_subplot(2, 1, 2)
ymax = 1.1 * coast.magnitude.max()
ax2.set_ylim([0, ymax])
ax2.plot(coast.magnitude, marker='o', color='red', linestyle='None', label='manual')
ax2.text(0.01, 0.98,'Within 26.5 km (distance to coast)', ha='left', va='top', transform=ax2.transAxes, fontsize=14)
ax2.set_ylabel('Magnitude')

plt.savefig('park-coast_events.png', dpi=200)

**Geodetic Plots - enu for each site, all data and reporting year**

In [ ]:
od = 11 #filter kernel

names = ['dt', 'obs', 'err']

In [ ]:
#loop for each site
sites = ['NPLY.LI', 'PGKH.CG', 'PGNE.CG']
for site in sites:
  siteid = str.split(site, '.')[0]
  netid = str.split(site, '.')[1]
  
  url= 'https://fits.geonet.org.nz/observation?typeID=e&siteID='+siteid+'&networkID='+netid
  dfe = pd.read_csv(url, names=names, skiprows=1, parse_dates={"Datetime" : ['dt']})
  url= 'https://fits.geonet.org.nz/observation?typeID=n&siteID='+siteid+'&networkID='+netid
  dfn = pd.read_csv(url, names=names, skiprows=1, parse_dates={"Datetime" : ['dt']})
  url= 'https://fits.geonet.org.nz/observation?typeID=u&siteID='+siteid+'&networkID='+netid
  dfu = pd.read_csv(url, names=names, skiprows=1, parse_dates={"Datetime" : ['dt']})

  #only data up to end date (end of reporting period)
  dfe = dfe[dfe['Datetime']<end]
  dfn = dfn[dfn['Datetime']<end]
  dfu = dfu[dfu['Datetime']<end]

  #join dataframes, rename columns
  df = pd.concat([dfe, dfn, dfu], axis=1, join='inner')
  names2 = ['Datetime', 'obse', 'erre', 'dtn', 'obsn', 'errn', 'dtu', 'obsu', 'erru']
  df.columns = names2      #rename columns

  #reference observation columns to first value
  df['obse'] -= df['obse'][0]
  df['obsn'] -= df['obsn'][0]
  df['obsu'] -= df['obsu'][0]

  #pre-filter observations
  df['obsef'] = scipy.signal.medfilt(df['obse'],od)
  df['obsnf'] = scipy.signal.medfilt(df['obsn'],od)
  df['obsuf'] = scipy.signal.medfilt(df['obsu'],od)

  #ticks
  majorTick = mpl.dates.YearLocator(1)
  majorFormat = mpl.dates.DateFormatter('%Y')
  minorTick = mpl.dates.MonthLocator()

  #plot all data
  fig = plt.figure(figsize=(15, 7.5))
  fig.subplots_adjust(hspace=0.3)

  ax = fig.add_subplot(3, 1, 1)
  plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
  plt.plot(df['Datetime'], df['obsn'], marker='o', color='black', linestyle='None')
  #plt.errorbar(df['Datetime'], df['obsn'], yerr=df['errn'], capsize=0, linestyle='None')
  plt.plot(df['Datetime'], df['obsnf'], marker='None', color='red')
  #ax.set_xlim([startall,end])
  plt.ylabel('dn (mm)')
  title = (siteid+' daily position change (mm)')
  plt.title(title)

  ax = fig.add_subplot(3, 1, 2)
  plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
  plt.plot(df['Datetime'], df['obse'], marker='o', color='black', linestyle='None')
  #plt.errorbar(df['Datetime'], df['obse'], yerr=df['erre'], capsize=0, linestyle='None')
  plt.plot(df['Datetime'], df['obsef'], marker='None', color='red')
  plt.ylabel('de (mm)')

  ax = fig.add_subplot(3, 1, 3)
  plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
  plt.plot(df['Datetime'], df['obsu'], marker='o', color='black', linestyle='None')
  #plt.errorbar(df['Datetime'], df['obsu'], yerr=df['erru'], capsize=0, linestyle='None')
  plt.plot(df['Datetime'], df['obsuf'], marker='None', color='red')
  plt.ylabel('du (mm)')

  plt.tight_layout()    
  fig.savefig('taranaki_'+siteid+'_all.png', dpi=200)

  #only data for reporting period
  dfe = dfe[(dfe['Datetime']<end)&(dfe['Datetime']>=startyear)]
  dfn = dfn[(dfn['Datetime']<end)&(dfn['Datetime']>=startyear)]
  dfu = dfu[(dfu['Datetime']<end)&(dfu['Datetime']>=startyear)]

  #join dataframes, rename columns
  df = pd.concat([dfe, dfn, dfu], axis=1, join='inner')
  names2 = ['Datetime', 'obse', 'erre', 'dtn', 'obsn', 'errn', 'dtu', 'obsu', 'erru']
  df.columns = names2      #rename columns

  #reference observation columns to first value
  df.reset_index(inplace=True)
  df['obse'] -= df['obse'][0]
  df['obsn'] -= df['obsn'][0]
  df['obsu'] -= df['obsu'][0]

  #pre-filter observations
  df['obsef'] = scipy.signal.medfilt(df['obse'],od)
  df['obsnf'] = scipy.signal.medfilt(df['obsn'],od)
  df['obsuf'] = scipy.signal.medfilt(df['obsu'],od)

  #ticks
  majorTick = mpl.dates.YearLocator(1)
  majorFormat = mpl.dates.DateFormatter('%Y')
  minorTick = mpl.dates.MonthLocator()

  #plot all data
  fig = plt.figure(figsize=(15, 7.5))
  fig.subplots_adjust(hspace=0.3)

  ax = fig.add_subplot(3, 1, 1)
  plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
  plt.plot(df['Datetime'], df['obsn'], marker='o', color='black', linestyle='None')
  #plt.errorbar(df['Datetime'], df['obsn'], yerr=df['errn'], capsize=0, linestyle='None')
  plt.plot(df['Datetime'], df['obsnf'], marker='None', color='red')
  plt.ylabel('dn (mm)')
  title = (siteid+' daily position change (mm)')
  plt.title(title)

  ax = fig.add_subplot(3, 1, 2)
  plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
  plt.plot(df['Datetime'], df['obse'], marker='o', color='black', linestyle='None')
  #plt.errorbar(df['Datetime'], df['obse'], yerr=df['erre'], capsize=0, linestyle='None')
  plt.plot(df['Datetime'], df['obsef'], marker='None', color='red')
  plt.ylabel('de (mm)')

  ax = fig.add_subplot(3, 1, 3)
  plt.grid(b=True, which='major', color='b', linestyle='--', alpha=0.5)
  plt.plot(df['Datetime'], df['obsu'], marker='o', color='black', linestyle='None')
  #plt.errorbar(df['Datetime'], df['obsu'], yerr=df['erru'], capsize=0, linestyle='None')
  plt.plot(df['Datetime'], df['obsuf'], marker='None', color='red')
  plt.ylabel('du (mm)')

  plt.tight_layout()
  fig.savefig('taranaki_'+siteid+'_year.png', dpi=200)